In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score
from xgboost import XGBRegressor, XGBClassifier
from load import dataset

In [104]:
# setup
predict_week = 2
predict_season = 2020
today_localize = pd.to_datetime('today').tz_localize('US/Central') - pd.Timedelta(hours=3)

In [87]:
# Load
df = dataset(predict_season=predict_season, predict_week=predict_week, window_size=4, update_data=True, update_seasons=[pd.to_datetime('today').year])

In [88]:
# Clean Up
df[df.columns[df.columns.str.contains('home|Home')]] = df[df.columns[df.columns.str.contains(
    'home|Home')]].groupby('home_team', as_index=False, group_keys=False).apply(lambda x: x.fillna(x.mean()))

df[df.columns[df.columns.str.contains('away|Away')]] = df[df.columns[df.columns.str.contains(
    'away|Away')]].groupby('away_team', as_index=False, group_keys=False).apply(lambda x: x.fillna(x.mean()))

df = df.dropna(axis=1, thresh=int(len(df) * .9))

df = df.fillna(0)

df[df.select_dtypes('bool').columns] = df.select_dtypes('bool').astype('int')

In [89]:
# Model
ct = make_column_transformer(
    (OrdinalEncoder(), ['season_type', 'home_conference', 'away_conference']),
    remainder='passthrough'
)

drop_columns =  ['game_id', 'season', 'venue', 'start_date', 'home_team', 'away_team', 'start_date', 'spread_target']

X_train = df[(df.start_date.dt.tz_convert('US/Central') < today_localize)].drop(drop_columns, axis=1)  
y_train = df[(df.start_date.dt.tz_convert('US/Central') < today_localize)][['spread_target']] 

X_pred = df[(df.start_date.dt.tz_convert('US/Central') >= today_localize)].drop(drop_columns, axis=1) 
# y_pred = y[X.start_date.dt.date >= pd.to_datetime('today')]

if len(X_pred) > 0:

    spread_model = XGBRegressor(learning_rate=0.1, 
                                    colsample_bytree=1.0,
                                    gamma=0.5,
                                    max_depth=5,
                                    min_child_weight=8,
                                    n_estimators=100,
                                    subsample=1.0)

    spread_model.fit(ct.fit_transform(X_train), y_train['spread_target'])
    y_spread_pred = np.around(spread_model.predict(ct.fit_transform(X_pred))/.5, decimals=0)*.5

    final_columns = ['game_id', 'season', 'week', 'season_type', 'home_team',
                    'home_conference', 'away_team', 'away_conference',  'pre_game_spread',
                    'predicted_spread']

    X_pred = pd.concat([
                        X_pred, 
                        df[df.start_date.dt.tz_convert('US/Central') >= today_localize].drop([col for col in df.columns if col not in drop_columns], axis=1) 
                        ], 
                        axis=1)

    predict_df = pd.concat([
            X_pred.reset_index(drop=True), 
            pd.Series(y_spread_pred, name='predicted_spread'),
        ], axis=1)

    predict_df = predict_df[final_columns]

    predict_df['spread_pick'] = np.where(predict_df['predicted_spread'] <= predict_df['pre_game_spread'], predict_df['home_team'], predict_df['away_team'])
    predict_df['straight_pick'] = np.where(predict_df['predicted_spread'] < 0, predict_df['home_team'], predict_df['away_team'])

    predict_df = predict_df[predict_df['week'] == predict_week]
    predict_df

else:
    print('Nothing to Predict')
    pass


In [90]:
predict_df

,game_id,season,week,season_type,home_team,home_conference,away_team,away_conference,pre_game_spread,predicted_spread,spread_pick,straight_pick
0,401236658,2020,2,regular,Kansas,Big 12,Coastal Carolina,Sun Belt,-6.5,-8.0,Kansas,Kansas
1,401236041,2020,2,regular,Oklahoma State,Big 12,Tulsa,American Athletic,-22.0,-23.0,Oklahoma State,Oklahoma State


In [91]:
# Pull in games that have occurred within the current week to append to the predictions
filepath = Path('../zillion_picks/picks')

result_dfs = []

for file in filepath.rglob('*.csv'):
    result_df = pd.read_csv(file)


    result_dfs.append(result_df)

result_df = pd.concat(result_dfs)

result_df = result_df[(result_df['week'] == predict_week) & (result_df['season'] == predict_season)]

try:
    result_df = result_df[~result_df.game_id.isin(predict_df.game_id)]
    result_df = result_df.drop(['actual_spread', 'spread_result', 'straight_result', 'start_date'], axis=1)

except:
        result_df = result_df.drop(['actual_spread', 'spread_result', 'straight_result', 'start_date'], axis=1)

In [92]:
result_df

,game_id,season,week,season_type,home_team,home_conference,away_team,away_conference,pre_game_spread,predicted_spread,spread_pick,straight_pick
0,401234559,2020,2,regular,Miami,ACC,UAB,Conference USA,-14.5,-20.5,Miami,Miami
1,401236081,2020,2,regular,Appalachian State,Sun Belt,Charlotte,Conference USA,-16.5,-22.5,Appalachian State,Appalachian State
2,401234565,2020,2,regular,North Carolina,ACC,Syracuse,ACC,-24.0,-18.5,Syracuse,North Carolina
3,401236221,2020,2,regular,Iowa State,Big 12,Louisiana,Sun Belt,-13.5,-17.0,Iowa State,Iowa State
4,401236038,2020,2,regular,West Virginia,Big 12,Eastern Kentucky,MISSING,-41.0,-44.0,West Virginia,West Virginia
5,401236053,2020,2,regular,Kansas State,Big 12,Arkansas State,Sun Belt,-14.5,-14.5,Kansas State,Kansas State
6,401240296,2020,2,regular,Baylor,Big 12,Louisiana Tech,Conference USA,-18.5,-21.5,Baylor,Baylor
7,401240113,2020,2,regular,Army,FBS Independents,Louisiana Monroe,Sun Belt,-24.5,-16.5,Louisiana Monroe,Army
8,401234574,2020,2,regular,Notre Dame,FBS Independents,Duke,ACC,-21.5,-21.0,Duke,Notre Dame
9,401207113,2020,2,regular,Texas State,Sun Belt,UT San Antonio,Conference USA,-6.0,-4.5,UT San Antonio,Texas State


In [93]:
# Merge Predit and Result Dataframes
try:
    predict_df = pd.concat([result_df, predict_df])
except:
    predict_df = result_df

In [94]:
# Add spread target in case the game is complete
predict_df = predict_df.merge(df[['game_id', 'spread_target', 'start_date']], on='game_id', how='left')

In [95]:
predict_df

,game_id,season,week,season_type,home_team,home_conference,away_team,away_conference,pre_game_spread,predicted_spread,spread_pick,straight_pick,spread_target,start_date
0,401234559,2020,2,regular,Miami,ACC,UAB,Conference USA,-14.5,-20.5,Miami,Miami,-17.0,2020-09-11 00:00:00+00:00
1,401236081,2020,2,regular,Appalachian State,Sun Belt,Charlotte,Conference USA,-16.5,-22.5,Appalachian State,Appalachian State,-15.0,2020-09-12 16:00:00+00:00
2,401234565,2020,2,regular,North Carolina,ACC,Syracuse,ACC,-24.0,-18.5,Syracuse,North Carolina,-25.0,2020-09-12 16:00:00+00:00
3,401236221,2020,2,regular,Iowa State,Big 12,Louisiana,Sun Belt,-13.5,-17.0,Iowa State,Iowa State,17.0,2020-09-12 16:00:00+00:00
4,401236038,2020,2,regular,West Virginia,Big 12,Eastern Kentucky,MISSING,-41.0,-44.0,West Virginia,West Virginia,-46.0,2020-09-12 16:00:00+00:00
5,401236053,2020,2,regular,Kansas State,Big 12,Arkansas State,Sun Belt,-14.5,-14.5,Kansas State,Kansas State,4.0,2020-09-12 16:00:00+00:00
6,401240296,2020,2,regular,Baylor,Big 12,Louisiana Tech,Conference USA,-18.5,-21.5,Baylor,Baylor,0.0,2020-09-12 16:00:00+00:00
7,401240113,2020,2,regular,Army,FBS Independents,Louisiana Monroe,Sun Belt,-24.5,-16.5,Louisiana Monroe,Army,-30.0,2020-09-12 17:30:00+00:00
8,401234574,2020,2,regular,Notre Dame,FBS Independents,Duke,ACC,-21.5,-21.0,Duke,Notre Dame,-14.0,2020-09-12 18:30:00+00:00
9,401207113,2020,2,regular,Texas State,Sun Belt,UT San Antonio,Conference USA,-6.0,-4.5,UT San Antonio,Texas State,3.0,2020-09-12 19:30:00+00:00


In [96]:
# Convert spread_target games that have not occurred to Null
predict_df['spread_target']  = np.where(pd.to_datetime(predict_df['start_date']).dt.tz_convert('US/Central') >= today_localize, np.nan, predict_df['spread_target'] )

In [97]:
predict_df

,game_id,season,week,season_type,home_team,home_conference,away_team,away_conference,pre_game_spread,predicted_spread,spread_pick,straight_pick,spread_target,start_date
0,401234559,2020,2,regular,Miami,ACC,UAB,Conference USA,-14.5,-20.5,Miami,Miami,-17.0,2020-09-11 00:00:00+00:00
1,401236081,2020,2,regular,Appalachian State,Sun Belt,Charlotte,Conference USA,-16.5,-22.5,Appalachian State,Appalachian State,-15.0,2020-09-12 16:00:00+00:00
2,401234565,2020,2,regular,North Carolina,ACC,Syracuse,ACC,-24.0,-18.5,Syracuse,North Carolina,-25.0,2020-09-12 16:00:00+00:00
3,401236221,2020,2,regular,Iowa State,Big 12,Louisiana,Sun Belt,-13.5,-17.0,Iowa State,Iowa State,17.0,2020-09-12 16:00:00+00:00
4,401236038,2020,2,regular,West Virginia,Big 12,Eastern Kentucky,MISSING,-41.0,-44.0,West Virginia,West Virginia,-46.0,2020-09-12 16:00:00+00:00
5,401236053,2020,2,regular,Kansas State,Big 12,Arkansas State,Sun Belt,-14.5,-14.5,Kansas State,Kansas State,4.0,2020-09-12 16:00:00+00:00
6,401240296,2020,2,regular,Baylor,Big 12,Louisiana Tech,Conference USA,-18.5,-21.5,Baylor,Baylor,0.0,2020-09-12 16:00:00+00:00
7,401240113,2020,2,regular,Army,FBS Independents,Louisiana Monroe,Sun Belt,-24.5,-16.5,Louisiana Monroe,Army,-30.0,2020-09-12 17:30:00+00:00
8,401234574,2020,2,regular,Notre Dame,FBS Independents,Duke,ACC,-21.5,-21.0,Duke,Notre Dame,-14.0,2020-09-12 18:30:00+00:00
9,401207113,2020,2,regular,Texas State,Sun Belt,UT San Antonio,Conference USA,-6.0,-4.5,UT San Antonio,Texas State,3.0,2020-09-12 19:30:00+00:00


In [98]:
# Spread Result Calculation
won = ((predict_df['predicted_spread'] > predict_df['pre_game_spread']) & (predict_df['spread_target'].fillna(0) > predict_df['pre_game_spread'])) | (
    (predict_df['predicted_spread'] <= predict_df['pre_game_spread']) & (predict_df['spread_target'].fillna(0) < predict_df['pre_game_spread']))

push = predict_df['spread_target'].fillna(0) == predict_df['pre_game_spread']

conditions = [won, push]
choices = ['Won', 'Push']

predict_df['spread_result'] = np.select(condlist=conditions, choicelist=choices, default='Lost')
predict_df['spread_result'] = np.where(predict_df['spread_target'].isnull(), np.nan, predict_df['spread_result'])

In [99]:
# Straight Up Result Calculation
result = predict_df['spread_target'].fillna(1) * predict_df['predicted_spread']

predict_df['straight_result'] = np.where(result > 0, 'Won', 'Lost')
predict_df['straight_result'] = np.where(predict_df['spread_target'].isnull(), np.nan, predict_df['straight_result'])

In [100]:
# Rename Column
predict_df = predict_df.rename(columns={'spread_target':'actual_spread'})

In [101]:
# predict_df['week'] = predict_df.week.astype(int)
predict_df

,game_id,season,week,season_type,home_team,home_conference,away_team,away_conference,pre_game_spread,predicted_spread,spread_pick,straight_pick,actual_spread,start_date,spread_result,straight_result
0,401234559,2020,2,regular,Miami,ACC,UAB,Conference USA,-14.5,-20.5,Miami,Miami,-17.0,2020-09-11 00:00:00+00:00,Won,Won
1,401236081,2020,2,regular,Appalachian State,Sun Belt,Charlotte,Conference USA,-16.5,-22.5,Appalachian State,Appalachian State,-15.0,2020-09-12 16:00:00+00:00,Lost,Won
2,401234565,2020,2,regular,North Carolina,ACC,Syracuse,ACC,-24.0,-18.5,Syracuse,North Carolina,-25.0,2020-09-12 16:00:00+00:00,Lost,Won
3,401236221,2020,2,regular,Iowa State,Big 12,Louisiana,Sun Belt,-13.5,-17.0,Iowa State,Iowa State,17.0,2020-09-12 16:00:00+00:00,Lost,Lost
4,401236038,2020,2,regular,West Virginia,Big 12,Eastern Kentucky,MISSING,-41.0,-44.0,West Virginia,West Virginia,-46.0,2020-09-12 16:00:00+00:00,Won,Won
5,401236053,2020,2,regular,Kansas State,Big 12,Arkansas State,Sun Belt,-14.5,-14.5,Kansas State,Kansas State,4.0,2020-09-12 16:00:00+00:00,Won,Lost
6,401240296,2020,2,regular,Baylor,Big 12,Louisiana Tech,Conference USA,-18.5,-21.5,Baylor,Baylor,0.0,2020-09-12 16:00:00+00:00,Lost,Lost
7,401240113,2020,2,regular,Army,FBS Independents,Louisiana Monroe,Sun Belt,-24.5,-16.5,Louisiana Monroe,Army,-30.0,2020-09-12 17:30:00+00:00,Lost,Won
8,401234574,2020,2,regular,Notre Dame,FBS Independents,Duke,ACC,-21.5,-21.0,Duke,Notre Dame,-14.0,2020-09-12 18:30:00+00:00,Won,Won
9,401207113,2020,2,regular,Texas State,Sun Belt,UT San Antonio,Conference USA,-6.0,-4.5,UT San Antonio,Texas State,3.0,2020-09-12 19:30:00+00:00,Won,Lost


In [102]:
# Export
filepath = Path('../zillion_picks/picks')

for key, group in predict_df.groupby(['season', 'week', 'season_type']):
    if group.season_type.unique() == 'postseason':
        group.to_csv(filepath/f'premium/{key[0]}_postseason.csv', index=False)
    else:
        group.to_csv(filepath/f'premium/{key[0]}_{str(key[1]).rjust(2, "0")}.csv', index=False)

In [103]:
important_features = pd.Series(data=spread_model.feature_importances_, index=X_train.columns)
important_features.sort_values(ascending=False,inplace=True)
important_features

pre_game_spread                    0.092599
rushingYardsAway                   0.005658
commits_All PositionsHome          0.005430
averageStars_Defensive BackHome    0.005231
firstDownsHome                     0.004877
                                     ...   
interceptionTDsHome                0.000000
commits_Defensive BackHome         0.000000
pre_game_away_win_prob             0.000000
pre_game_home_win_prob             0.000000
week                               0.000000
Length: 301, dtype: float32